# TORONTO NEIGHBORHOODS

This notebook is for week 3 of the Applied Data Science Capstone project. In this notebook, I will be scraping the wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M. 

This data will give a dataframe of the neighborhoods of Toronto.

## Section 1 - Capturing the data from Wikipedia.


For this section I decided to experiment with beautiful soup for the scraping.

In [2]:
import pandas as pd #for data analysis
import numpy as np #import numpy
!pip install beautifulsoup4 #for scraping the Wiki

from bs4 import BeautifulSoup as bs

In [3]:
#download data from wikipedia
!wget -q -O 'List_of_postal_codes_of_Canada:_M' https://en.wikipedia.org/wiki/ 

print("data downloaded")

data downloaded


In [4]:
with open("index.html") as fp:  #read page into BeautifulSoup
    wiki = bs(fp)

table = wiki.table #extract table section of wiki page

In [5]:
n_columns = 0
n_rows=0
column_names = []
    
# Define my data frame
 #Count all html row tags in order to find n_rows
for row in table.find_all('tr'):
    td_tags = row.find_all('td')
    if len(td_tags) > 0:
       n_rows+=1 
       if n_columns == 0:
            n_columns = len(td_tags)
    th_tags = row.find_all('th')  # count the number of header tags to find n_columns
    if len(th_tags) > 0 and len(column_names) == 0:
        for th in th_tags:
           column_names.append(th.get_text().rstrip())
    
columns = column_names if len(column_names) > 0 else range(0,n_columns)
#build my data frame
neighborhoods = pd.DataFrame(columns = columns, index= range(0,n_rows))
row_marker = 0
for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        neighborhoods.iat[row_marker,column_marker] = (column.get_text()).rstrip()
        column_marker += 1
    if len(columns) > 0:
        row_marker += 1
        
neighborhoods.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
#Clean and normalize my data
 # Remove Not assigned Postal Codes
neighborhoods=neighborhoods[neighborhoods.Borough != 'Not assigned']
#Check for unnamed neighborhoods
check_df = neighborhoods[neighborhoods.Neighborhood == ""]
check_df


,Postal Code,Borough,Neighborhood


I notice that there are no unnamed neighborhoods, so nothing need to be done here.

In [8]:
#After removing rows, need to reset the index.
neighborhoods.reset_index(drop=True, inplace=True)

In [9]:
neighborhoods.head(103)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [10]:
neighborhoods.shape

(103, 3)

### THIS IS THE END OF SECTION 1

## Section 2 - Find Latitude and Longitude for Toronto postal codes.

In this section I will attempt to use geocoder to gather lat/lon data on Toronto postal codes. 

In [11]:
!pip install geocoder

I attempted to use geocoder, but the script below took an inordinate amount of time. I have instead decided to default to the provided data set.

In [12]:
# Use geocoder
#import geocoder # import geocoder
#for ind in neighborhoods.index:  
#    postal_code = neighborhoods['Postal Code'][ind]
#    # initialize your variable to None
#    lat_lng_coords = None
#    
#    # loop until you get the coordinates
#    while(lat_lng_coords is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#        lat_lng_coords = g.latlng

#    neighborhoods['Latitude'][ind] = lat_lng_coords[0]
#    neighborhoods['Longitude'][ind] = lat_lng_coords[1]

Importing Dataset.

In [13]:
!wget -q -O "Geospatial_data.csv" http://cocl.us/Geospatial_data

Import the data to a data frame.

In [14]:
loc_df = pd.read_csv("Geospatial_data.csv")

In [15]:
loc_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
#Join the two data frames together.
neighborhoods= pd.merge(neighborhoods, loc_df, on='Postal Code')
neighborhoods.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


### END OF SECTION 2

## SECTION 3 - MAPPING

In this section I will be clustering the neighborhoods of Toronto in order to find similarities and disparities.

In [17]:
#Import necessary libraries and handlers
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


Get geocoords of Toronto, CA

In [18]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [19]:
# create map of Toronto neighborhoods using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [20]:
neighborhoods.Borough.value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

Since there are not nearly as many neighborhoods as there were in the New York assignment. I have decided not to divide the boroughs out. Instead I will run the clustering on all boroughs simultaneously.

Define foursquare variables.

In [21]:
CLIENT_ID = 'RQAZZWMEAEE0MNNHTRGZA55KKFA0VLWPXF0HDBBIZZLKWX0J' # your Foursquare ID
CLIENT_SECRET = 'ELEBUPLGHNG3ANIKXFH1M40Y015B4KFFD3IGTSIV2O2PETMR' # your Foursquare Secret
ACCESS_TOKEN = 'MWQW1FCJIQNAG4QHD0DM4V5VD0T5GV3YKJOHXPM5ZVUZNBPD' #oauthorization token
VERSION = '20200527' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('ACCESS_TOKEN:' + ACCESS_TOKEN)

Your credentails:
CLIENT_ID: RQAZZWMEAEE0MNNHTRGZA55KKFA0VLWPXF0HDBBIZZLKWX0J
CLIENT_SECRET:ELEBUPLGHNG3ANIKXFH1M40Y015B4KFFD3IGTSIV2O2PETMR
ACCESS_TOKEN:MWQW1FCJIQNAG4QHD0DM4V5VD0T5GV3YKJOHXPM5ZVUZNBPD


Define getNearbyVenues script in order to find popular venues in and around each neighborhood. Number of venues is limited to 100 within a 500 m radius.

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
#Use the getNearbyVenues script to cycle through all of the neighborhoods
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [25]:
print(toronto_venues.shape)
toronto_venues.head()

(2132, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,649 Variety,43.754513,-79.331942,Convenience Store
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [26]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Wilson Heights, Downsview North",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
...,...,...,...,...,...,...
"Willowdale, Willowdale West",5,5,5,5,5,5
Woburn,3,3,3,3,3,3
Woodbine Heights,9,9,9,9,9,9


In [27]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 unique categories.


In [28]:
# Use one hot encoding to change categorical values to binaries.
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
toronto_onehot.shape

(2132, 272)

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
91,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
92,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0
93,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
toronto_grouped.shape

(95, 272)

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Sandwich Place,Athletics & Sports,Pub,Coffee Shop,Pool,Skating Rink,Gym,Doner Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Diner,Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Ice Cream Shop,Deli / Bodega,Fried Chicken Joint,Sandwich Place
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Women's Store,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",Restaurant,Coffee Shop,Italian Restaurant,Sandwich Place,Comfort Food Restaurant,Thai Restaurant,Café,Sushi Restaurant,Liquor Store,Pub


In [34]:
neighborhoods_venues_sorted.shape

(95, 11)

In [35]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [36]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
#Because some neighborhoods have more than one Postal Code, the toronto_venues grouped them together.
#So after merging the data there are a few neighborhoods with no common venues. Need to drop those rows.

toronto_merged.dropna(inplace=True)
toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Park,Convenience Store,Food & Drink Shop,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Pizza Place,French Restaurant,Coffee Shop,Portuguese Restaurant,Hockey Arena,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Bakery,Pub,Park,Theater,Breakfast Spot,Café,Restaurant,Yoga Studio,Shoe Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Accessories Store,Women's Store,Gift Shop,Boutique,Miscellaneous Shop,Event Space,Coffee Shop,Furniture / Home Store,Vietnamese Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Yoga Studio,Music Venue,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,College Cafeteria


In [37]:
toronto_merged = toronto_merged.astype({"Cluster Labels": int})
toronto_merged.shape

(99, 16)

In [38]:
# Let's create a map showing the clusters by color.
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Explore the clustered data.

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,East York,0,Pizza Place,Pharmacy,Gastropub,Intersection,Bank,Athletics & Sports,Fast Food Restaurant,Pet Store,Gym / Fitness Center,Dance Studio
10,North York,0,Pizza Place,Pub,Asian Restaurant,Sushi Restaurant,Japanese Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
50,North York,0,Pizza Place,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Gym / Fitness Center
70,Etobicoke,0,Pizza Place,Coffee Shop,Intersection,Chinese Restaurant,Middle Eastern Restaurant,Discount Store,Sandwich Place,Doner Restaurant,Dog Run,Department Store
72,North York,0,Pizza Place,Home Service,Coffee Shop,Bank,Pharmacy,Gluten-free Restaurant,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore
77,Etobicoke,0,Pizza Place,Park,Mobile Phone Shop,Sandwich Place,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Deli / Bodega,Distribution Center
82,Scarborough,0,Pharmacy,Pizza Place,Convenience Store,Italian Restaurant,Fried Chicken Joint,Thai Restaurant,Chinese Restaurant,Intersection,Gas Station,Bank
93,Etobicoke,0,Pizza Place,Pharmacy,Sandwich Place,Athletics & Sports,Pub,Coffee Shop,Pool,Skating Rink,Gym,Doner Restaurant


In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,Pizza Place,French Restaurant,Coffee Shop,Portuguese Restaurant,Hockey Arena,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,Downtown Toronto,1,Coffee Shop,Bakery,Pub,Park,Theater,Breakfast Spot,Café,Restaurant,Yoga Studio,Shoe Store
3,North York,1,Clothing Store,Accessories Store,Women's Store,Gift Shop,Boutique,Miscellaneous Shop,Event Space,Coffee Shop,Furniture / Home Store,Vietnamese Restaurant
4,Downtown Toronto,1,Coffee Shop,Yoga Studio,Music Venue,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,College Cafeteria
6,Scarborough,1,Fast Food Restaurant,Print Shop,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
...,...,...,...,...,...,...,...,...,...,...,...,...
96,Downtown Toronto,1,Coffee Shop,Bakery,Market,Café,Restaurant,Pub,Pizza Place,Park,Italian Restaurant,Japanese Restaurant
97,Downtown Toronto,1,Coffee Shop,Café,Japanese Restaurant,Hotel,Gym,Restaurant,Seafood Restaurant,Salad Place,Steakhouse,Deli / Bodega
99,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Yoga Studio,Café,Bubble Tea Shop,Pub,Hotel
100,East Toronto,1,Light Rail Station,Yoga Studio,Garden Center,Restaurant,Smoke Shop,Auto Workshop,Burrito Place,Fast Food Restaurant,Farmers Market,Garden


In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2,Park,Convenience Store,Food & Drink Shop,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
21,York,2,Park,Women's Store,Spa,Gourmet Shop,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
35,East York,2,Park,Coffee Shop,Convenience Store,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
61,Central Toronto,2,Park,Bus Line,Swim School,Women's Store,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
64,York,2,Park,Convenience Store,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
66,North York,2,Park,Bank,Convenience Store,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
83,Central Toronto,2,Park,Playground,Tennis Court,Summer Camp,Women's Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
85,Scarborough,2,Playground,Park,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
91,Downtown Toronto,2,Park,Playground,Trail,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
98,Etobicoke,2,Park,River,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant


In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,3,Bar,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Farmers Market


In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,4,Furniture / Home Store,Baseball Field,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Department Store
101,Etobicoke,4,Construction & Landscaping,Baseball Field,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore


## The above clustering data helps to understand what neighborhoods have similar amenities. Of course, if someone was looking to move from one to another, there would be a lot more data that was necessary (crime rate, school zone, etc.), but this is a start.